<a href="https://colab.research.google.com/github/thisisSHAX/ML_Ops_Project-/blob/main/Autism_Screening_Prediction_App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Creating a web app to deploy the model

In [ ]:
!pip install wandb
!pip install onnxruntime
!pip install -q gradio 

     |████████████████████████████████| 1.7 MB 5.2 MB/s 
     |████████████████████████████████| 180 kB 53.2 MB/s 
     |████████████████████████████████| 140 kB 60.9 MB/s 
     |████████████████████████████████| 97 kB 6.5 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-py3-none-any.whl size=6502 sha256=467b053c1bf6276c925e95e579fb84d3f0143feef964edb74d44050040a7952a
  Stored in directory: /root/.cache/pip/wheels/50/ca/fa/8fca8d246e64f19488d07567547ddec8eb084e8c0d7a59226a
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=b9c3ff68648f11e66ed2784f49888b9d65bf02612f7f422e5c63fc8ece30402d
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built subprocess32 pathtools
     |████████████████████████████████| 4.8 MB 5.5 MB/s 
     |████████████████████████████████| 979 kB 5.2 MB/s 
     |███████████████████████

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
import os
import wandb
import pandas as pd
import numpy as np
import onnxruntime as rt
from gradio import gradio as gr

### Initialise weights and biases

In [ ]:
os.environ["WANDB_API_KEY"] = "3ee9089935c4340e01c38077092a69fb10a6f6ac"

In [ ]:
run = wandb.init(project='autism_screening')

wandb: Currently logged in as: maytune (use `wandb login --relogin` to force relogin)


### Download the model and load it

In [ ]:
ARTIFACT_NAME = 'LogReg_AutismScreening:v0'

In [ ]:
artifact = run.use_artifact('maytune/autism_screening/' + ARTIFACT_NAME, type='model')
artifact_dir = artifact.download()

In [ ]:
artifact_dir

'./artifacts/LogReg_AutismScreening:v0'

In [ ]:
!ls -al  artifacts/LogReg_AutismScreening:v0

total 16
drwxr-xr-x 2 root root 4096 Nov 26 17:32 .
drwxr-xr-x 3 root root 4096 Nov 26 17:32 ..
-rw-r--r-- 1 root root 5380 Nov 26 17:32 autismscreening_logreg.onnx


### Columns

In [ ]:
x_columns = ['A1','A2','A3','A4','A5','A6','A7','A8','A9','A10','Age_Mons','Sex','Ethnicity','Jaundice','Family_mem_with_ASD']

In [ ]:
cat_features = ['A1','A2','A3','A4','A5','A6','A7','A8','A9','A10','Sex','Ethnicity','Jaundice','Family_mem_with_ASD']

In [ ]:
num_features = list(set(x_columns) - set(cat_features))

### Implement the predict() function

In [ ]:
def predict_autism(A1, A2, A3,
                  A4, A5, A6, 
                  A7, A8, A9, 
                  A10, Age_Mons, Sex, Ethnicity, Jaundice, Family_mem_with_ASD ):
    if(A1=="Always" or A1=="Usually"):
      A1_num = 0
    else:
      A1_num = 1
    if(A2=="Always" or A2=="Usually"):
      A2_num = 0
    else:
      A2_num = 1
    if(A3=="Always" or A3=="Usually"):
      A3_num = 0
    else:
      A3_num = 1
    if(A4=="Always" or A4=="Usually"):
      A4_num = 0
    else:
      A4_num = 1
    if(A5=="Always" or A5=="Usually"):
      A5_num = 0
    else:
      A5_num = 1
    if(A6=="Always" or A6=="Usually"):
      A6_num = 0
    else:
      A6_num = 1
    if(A7=="Always" or A7=="Usually"):
      A7_num = 0
    else:
      A7_num = 1
    if(A8=="Always" or A8=="Usually"):
      A8_num = 0
    else:
      A8_num = 1
    if(A9=="Always" or A9=="Usually"):
      A9_num = 0
    else:
      A9_num = 1
    if(A10=="Always" or A10=="Usually" or A10=="Sometimes"):
      A10_num = 1
    else:
      A10_num = 0
    inputs_dict = {'A1' :np.float32(A1_num) , 
              'A2': np.float32(A2_num), 
              'A3': np.float32(A3_num), 
              'A4': np.float32(A4_num), 
              'A5': np.float32(A5_num), 
              'A6': np.float32(A6_num), 
              'A7': np.float32(A7_num), 
              'A8': np.float32(A8_num), 
              'A9': np.float32(A9_num), 
              'A10': np.float32(A10_num), 
              'Age_Mons' : float(Age_Mons),
              'Sex': Sex, 
              'Ethnicity': Ethnicity,
              'Jaundice': Jaundice,
              'Family_mem_with_ASD': Family_mem_with_ASD}

    df = pd.DataFrame(inputs_dict, index = [0])
    print(df)

    inputs = {c: df[c].values for c in df.columns}
    for c in num_features:
        inputs[c] = inputs[c].astype(np.float32)
    for k in inputs:
        inputs[k] = inputs[k].reshape((inputs[k].shape[0], 1))            
  
    sess = rt.InferenceSession(artifact_dir + '/autismscreening_logreg.onnx')
    pred_onx = sess.run(None, inputs)
    predicted = pred_onx[0][0]
    if(predicted == 0):
      resultstr = "Child may not have autism"
    else:
      resultstr = "Child may have autism"  
    return 'Autism prediction: ' + resultstr

### Implement UI

In [ ]:
Age_Mons = gr.inputs.Slider(minimum=0, maximum=36, label="Age of infant (in months)")

A1 = gr.inputs.Radio(["Always", "Usually", "Sometimes", "Rarely", "Never"], default="Always", label="Does your child look at you when you call his/her name?")
A2 = gr.inputs.Radio(["Always", "Usually", "Sometimes", "Rarely", "Never"], default="Always", label="Is it easy for you to get eye-contact with your child?")
A3 = gr.inputs.Radio(["Always", "Usually", "Sometimes", "Rarely", "Never"], default="Always", label="Does your child point to indicate that he/she wants something?")
A4 = gr.inputs.Radio(["Always", "Usually", "Sometimes", "Rarely", "Never"], default="Always", label="Does your child point to share interest with you?")
A5 = gr.inputs.Radio(["Always", "Usually", "Sometimes", "Rarely", "Never"], default="Always", label="Does your child pretend?")
A6 = gr.inputs.Radio(["Always", "Usually", "Sometimes", "Rarely", "Never"], default="Always", label="Does your child follow where you are looking?")
A7 = gr.inputs.Radio(["Always", "Usually", "Sometimes", "Rarely", "Never"], default="Always", label="If you or someone else in your family is visibly upset, Does your child show signs of wanting to comfort them?")
A8 = gr.inputs.Radio(["Always", "Usually", "Sometimes", "Rarely", "Never"], default="Always", label="Does your child use simple words?")
A9 = gr.inputs.Radio(["Always", "Usually", "Sometimes", "Rarely", "Never"], default="Always", label="Does your child use simple gestures")
A10 = gr.inputs.Radio(["Always", "Usually", "Sometimes", "Rarely", "Never"], default="Never", label="Does your child stare at nothing with no apparent purpose?")

Sex = gr.inputs.Radio(['m', 'f'], default="m", label="Sex")
Ethnicity = gr.inputs.Dropdown(['asian', 'black', 'Hispanic', 'Latino', 'middle eastern', 'mixed', 'Native Indian', 'Others', 'Pacifica', 'south asian', 'White European'], default="Others", label="Ethnicity")
Jaundice = gr.inputs.Radio(['no', 'yes'], default="no", label="Was the child born with Jaundice?")
Family_mem_with_ASD = gr.inputs.Radio(['no', 'yes'], default="no", label="Does any immediate family member have ASD?")

gr.Interface(predict_autism, [A1, A2, A3,
                  A4, A5, A6, 
                  A7, A8, A9, 
                  A10, Age_Mons, Sex, Ethnicity, Jaundice, Family_mem_with_ASD], "text", live=False).launch(debug=True);

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://15138.gradio.app

This share link will expire in 72 hours. To get longer links, send an email to: support@gradio.app


    A1   A2   A3   A4   A5  ...  Age_Mons  Sex  Ethnicity  Jaundice  Family_mem_with_ASD
0  0.0  0.0  0.0  0.0  0.0  ...      12.9    m      black       yes                  yes

[1 rows x 15 columns]
    A1   A2   A3   A4   A5  ...  Age_Mons  Sex  Ethnicity  Jaundice  Family_mem_with_ASD
0  0.0  0.0  1.0  1.0  1.0  ...      12.9    m      black       yes                  yes

[1 rows x 15 columns]
